In [1]:
pip install holiday

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# 合并平均酒店的部分，手動調整讀取的檔案，取得3，4，5星級的平均酒店價格趨勢

import pandas as pd
from sklearn.preprocessing import StandardScaler
import glob
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import grangercausalitytests
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import holidays
import statsmodels.api as sm

# 定義讀取文件的路徑和模式
folder_path = "C:/Users/winst/Downloads/data345/5starhotel"  # 替換為實際的文件夾路徑
file_pattern = folder_path + "/5_*.csv"

# 讀取所有符合條件的 CSV 文件
file_list = glob.glob(file_pattern)

# 初始化一個空列表用於存儲數據
data_frames = []

# 逐個讀取每個文件並存入列表
scaler = MinMaxScaler()
for file in file_list:
    df = pd.read_csv(file)
    #print(df.head())
    # 確保有 'date' 和 'price' 列，否則跳過
    df = df.dropna(subset=["price"])[["date", "price"]]
    #print(df.head())
    df["date"] = df["date"].replace({", departure date." :"", ", return date.": ""}, regex=True)
    df["price"] = df["price"].replace({",": "", "$": ""}, regex=True)
    df["price"] = df["price"].apply(lambda x: float(x[:-1]) * 1000 if x.endswith('K') else (float(x[:-1]) * 1000000 if x.endswith('M') else float(x)))
    df["price"] = scaler.fit_transform(df[["price"]])
    if "date" in df.columns and "price" in df.columns:
        data_frames.append(df)  # 保留必要列
        #print(data_frames)


# 合併所有數據框（以日期為基準）
merged_data = pd.concat(data_frames)
print(merged_data)

# 將日期轉換為日期格式，便於排序
merged_data["date"] = pd.to_datetime(merged_data["date"])

# 按日期排序
merged_data = merged_data.sort_values(by="date").reset_index(drop=True)

# 將每個酒店價格分組匯總
grouped_data = merged_data.groupby("date").mean(numeric_only=True).reset_index()

#print(grouped_data.columns)
# 標準化價格
scaler = MinMaxScaler()
grouped_data["standardized_price"] = scaler.fit_transform(grouped_data[["price"]])

# 顯示結果
print("合併後的數據:")
print(grouped_data)

# 保存結果到 CSV 文件
grouped_data.to_csv("merged_price_trend_5.csv", index=False)


In [ ]:
# 機票和酒店平均價格趨勢的對比，一樣手動調整讀取的檔案

# 1. 讀取數據集
# 機票價格數據
airline_data = pd.read_csv("airline_prices.csv")
# 酒店價格數據
hotel_data = pd.read_csv("merged_price_trend_5.csv")

# 檢查數據格式
print(airline_data.head())
print(hotel_data.head())

#hotel_data["date"] = hotel_data["date"].replace({", departure date." :"", ", return date.": ""}, regex=True)

# 2. 數據清洗與處理
# 機票數據清洗
airline_data["Date"] = pd.to_datetime(airline_data["Date"])
airline_data = airline_data.dropna(subset=["Prices"])

# 酒店數據清洗
hotel_data["date"] = pd.to_datetime(hotel_data["date"])
hotel_data = hotel_data.dropna(subset=["price"])

print(airline_data.head())
print(hotel_data.head())
airline_data["Prices"] = airline_data["Prices"].replace({",": "", "$": ""}, regex=True)
airline_data["Prices"] = airline_data["Prices"].str.replace(r'[^\d\.K]', '', regex=True)

# 2. 处理 K 和 M
def convert_price(value):
    # 如果以K结尾，转化为千
    if value.endswith('K'):
        return float(value[:-1]) * 1000
    # 如果以M结尾，转化为百万
    elif value.endswith('M'):
        return float(value[:-1]) * 1000000
    else:
        return float(value)  # 直接转换为float

# 应用转换函数
airline_data["Prices"] = airline_data["Prices"].apply(convert_price)

print(airline_data.head())
print(hotel_data.head())

# 聚合酒店價格數據（按日期計算平均價格）
hotel_prices_daily = hotel_data.groupby("date")["price"].mean().reset_index()
hotel_prices_daily.rename(columns={"date": "Date", "price": "Hotel_Prices"}, inplace=True)

# 合併數據集
merged_data = pd.merge(airline_data, hotel_prices_daily, on="Date", how="inner")
merged_data.rename(columns={"Prices": "Airline_Prices"}, inplace=True)

scaler = MinMaxScaler()
merged_data[["Airline_Prices", "Hotel_Prices"]] = scaler.fit_transform(merged_data[["Airline_Prices", "Hotel_Prices"]])

print("\n合併後的數據:")
print(merged_data.head())

# 3. 相關性分析
correlation = merged_data["Airline_Prices"].corr(merged_data["Hotel_Prices"])
print(f"\n機票價格與酒店價格的相關係數: {correlation}")

# 4. Granger 因果檢驗
print("\nGranger 因果檢驗結果:")
granger_result = grangercausalitytests(merged_data[["Airline_Prices", "Hotel_Prices"]], maxlag=3)

# 5. 可視化
plt.figure(figsize=(14, 7))
plt.plot(merged_data["Date"], merged_data["Airline_Prices"], label="Airline Prices", color="blue")
plt.plot(merged_data["Date"], merged_data["Hotel_Prices"], label="Hotel Prices", color="orange")
plt.xlabel("Date")
plt.ylabel("Prices")
plt.title("Airline Prices vs 5_star Hotel Prices")
plt.legend()
plt.show()


In [ ]:
# 酒店和酒店之間平均價格趨勢的對比，一樣手動調整讀取的檔案

# 1. 讀取數據集
# 機票價格數據
hotel_data1 = pd.read_csv("merged_price_trend_3.csv")
# 酒店價格數據
hotel_data = pd.read_csv("merged_price_trend_5.csv")

#hotel_data["date"] = hotel_data["date"].replace({", departure date." :"", ", return date.": ""}, regex=True)

# 2. 數據清洗與處理
# 酒店數據清洗
hotel_data1["date"] = pd.to_datetime(hotel_data["date"])
hotel_data1 = hotel_data1.dropna(subset=["price"])

# 酒店數據清洗
hotel_data["date"] = pd.to_datetime(hotel_data["date"])
hotel_data = hotel_data.dropna(subset=["price"])

# 聚合酒店價格數據（按日期計算平均價格）
hotel_prices_daily = hotel_data.groupby("date")["price"].mean().reset_index()
hotel_prices_daily.rename(columns={"date": "Date", "price": "Hotel_Prices"}, inplace=True)

hotel_prices_daily1 = hotel_data1.groupby("date")["price"].mean().reset_index()
hotel_prices_daily1.rename(columns={"date": "Date", "price": "Hotel_Prices1"}, inplace=True)

# 合併數據集
merged_data = pd.merge(hotel_prices_daily1, hotel_prices_daily, on="Date", how="inner")
merged_data.rename(columns={"Hotel_Prices1": "Airline_Prices"}, inplace=True)

scaler = MinMaxScaler()
merged_data[["Airline_Prices", "Hotel_Prices"]] = scaler.fit_transform(merged_data[["Airline_Prices", "Hotel_Prices"]])

print("\n合併後的數據:")
print(merged_data.head())

# 3. 相關性分析
correlation = merged_data["Airline_Prices"].corr(merged_data["Hotel_Prices"])
print(f"\n酒店價格的相關係數: {correlation}")

# 4. Granger 因果檢驗
print("\nGranger 因果檢驗結果:")
granger_result = grangercausalitytests(merged_data[["Airline_Prices", "Hotel_Prices"]], maxlag=3)

# 5. 可視化
plt.figure(figsize=(14, 7))
plt.plot(merged_data["Date"], merged_data["Airline_Prices"], label="3starHotel Prices", color="blue")
plt.plot(merged_data["Date"], merged_data["Hotel_Prices"], label="5starHotel Prices", color="orange")
plt.xlabel("Date")
plt.ylabel("Hotel_Prices1")
plt.title("Hotel_3 Prices vs Hotel_5 Prices")
plt.legend()
plt.show()


In [ ]:
# 最後分析，新增特徵、可視化等

# 要處理的文件列表
files = ["merged_price_trend_3.csv", "merged_price_trend_4.csv", "merged_price_trend_5.csv"]

# 存儲合併後的數據
merged_data = pd.DataFrame()

# 處理每個文件
for file in files:
    # 讀取數據
    data = pd.read_csv(file)
    
    # 提取檔名中的星級數字並新增一個特徵
    star_rating = int(file.split("_")[-1].split(".")[0])  # 提取數字部分
    data["Star"] = star_rating  # 添加星級列
    
    # 合併數據
    merged_data = pd.concat([merged_data, data], ignore_index=True)

# 將合併後的數據導出到新的 CSV 文件
merged_data.to_csv("merged_all_price_trends.csv", index=False)

# 1. 讀取數據集
airline_data = pd.read_csv("airline_prices.csv")
hotel_data = pd.read_csv("merged_all_price_trends.csv")

# 2. 數據清洗與處理
airline_data["Date"] = pd.to_datetime(airline_data["Date"])
airline_data = airline_data.dropna(subset=["Prices"])

hotel_data["date"] = pd.to_datetime(hotel_data["date"])
hotel_data = hotel_data.dropna(subset=["price"])

# 清洗機票價格
airline_data["Prices"] = airline_data["Prices"].replace({",": "", "$": ""}, regex=True)
airline_data["Prices"] = airline_data["Prices"].str.replace(r'[^\d\.K]', '', regex=True)

def convert_price(value):
    if value.endswith('K'):
        return float(value[:-1]) * 1000
    elif value.endswith('M'):
        return float(value[:-1]) * 1000000
    else:
        return float(value)

airline_data["Prices"] = airline_data["Prices"].apply(convert_price)

# 聚合酒店價格數據（按日期計算平均價格）
hotel_prices_daily = hotel_data.groupby("date")["price"].mean().reset_index()
hotel_prices_daily.rename(columns={"date": "Date", "price": "Hotel_Prices"}, inplace=True)

# 合併數據集
merged_data = pd.merge(airline_data, hotel_prices_daily, on="Date", how="inner")
merged_data.rename(columns={"Prices": "Airline_Prices"}, inplace=True)

# 設置季節（春季、夏季、秋季、冬季）
def get_season(date):
    if date.month in [3, 4, 5]:
        return "Spring"
    elif date.month in [6, 7, 8]:
        return "Summer"
    elif date.month in [9, 10, 11]:
        return "Autumn"
    elif date.month in [12,1,2]:
        return "Winter"

# 判斷是否為假期（包含假期前後幾天）
def is_holiday_extended(date, holidays_list, days=3):
    for holiday in holidays_list:
        if abs((holiday - date).days) <= days:
            return 1
    return 0

# 獲取 2024 年日本所有假期
jp_holidays = holidays.Japan(years=[2024])
holiday_dates = list(pd.Timestamp(holiday) for holiday in jp_holidays.keys())

# 添加新列，包含假期前後 3 天的範圍
merged_data['IsHoliday'] = merged_data['Date'].apply(
    lambda date: is_holiday_extended(date, holiday_dates, days=3)
)

# 添加新列
merged_data['Season'] = merged_data['Date'].apply(get_season)

# 進行One-Hot編碼
season_dummies = pd.get_dummies(merged_data['Season'], prefix='Season')

# 合併One-Hot編碼的列
merged_data = pd.concat([merged_data, season_dummies], axis=1)

# 刪除原始的 'Season' 列
merged_data = merged_data.drop(columns=["Season"])

# 將 One-Hot 編碼的季節列轉換為數值型（1 和 0）
season_columns = ['Season_Autumn', 'Season_Spring', 'Season_Summer', 'Season_Winter']
merged_data[season_columns] = merged_data[season_columns].astype(int)

# 4. 數據標準化
scaler = MinMaxScaler()
merged_data[["Airline_Prices", "Hotel_Prices"]] = scaler.fit_transform(merged_data[["Airline_Prices", "Hotel_Prices"]])

merged_data["isWeekend"] = merged_data["Date"].dt.weekday >= 5

# 5. 按季節和假期分析價格波動
seasonal_avg_prices = merged_data.groupby('Season_Spring')[['Airline_Prices', 'Hotel_Prices']].mean()
holiday_avg_prices = merged_data.groupby('IsHoliday')[['Airline_Prices', 'Hotel_Prices']].mean()

# 6. 線性回歸模型
# 定義自變量和因變量
#print(merged_data.dtypes)
X = merged_data[['Airline_Prices', 'isWeekend', 'IsHoliday', 'Season_Spring', 'Season_Summer', 'Season_Autumn']]
y = merged_data['Hotel_Prices']

# 1. 強制轉換 X 和 y 為數值型（並排除任何錯誤的字符）
X = X.apply(pd.to_numeric, errors='coerce')  # 使用 'coerce' 可以將無法轉換的值設為 NaN
y = pd.to_numeric(y, errors='coerce')  # 也強制轉換 y 為數值型

# 2. 檢查是否有空值，並填充空值（例如填充為 0）
# X = X.fillna(0)
# y = y.fillna(0)

# 3. 確保加入常數項
X = sm.add_constant(X)

X['isWeekend'] = X['isWeekend'].astype(int)

print("First few rows of X:")
# print(X.head())

print("First few rows of y:")
# print(y.head())
model = sm.OLS(y, X)
results = model.fit()

# 顯示回歸結果
print("\n線性回歸結果:")
print(results.summary())

# 7. Granger因果檢驗
print("\nGranger 因果檢驗結果:")
granger_result = grangercausalitytests(merged_data[["Airline_Prices", "Hotel_Prices"]], maxlag=10)

import seaborn as sns

# 添加季節名稱回去以便於分組
merged_data['Season'] = merged_data[['Season_Spring', 'Season_Summer', 'Season_Autumn', 'Season_Winter']].idxmax(axis=1)
merged_data['Season'] = merged_data['Season'].str.replace('Season_', '')

# 按季節畫酒店價格變動趨勢
plt.figure(figsize=(10, 6))
sns.boxplot(x="Season", y="Hotel_Prices", data=merged_data, order=['Spring', 'Summer', 'Autumn', 'Winter'])
plt.title("Hotelprices in different seasons", fontsize=16)
plt.xlabel("seasons", fontsize=12)
plt.ylabel("hotel prices(normalize)", fontsize=12)
plt.show()

# print(merged_data['isWeekend'].unique())
# print(merged_data['isWeekend'].isnull().sum())

# 确保 'Weekend_Label' 列的值正确
merged_data['Weekend_Label'] = merged_data['isWeekend'].map({True: "Weekend", False: "Weekday"})
#print(merged_data['isWeekend'] == 1)
# 检查是否有空值，并修复（如必要）
if merged_data['Weekend_Label'].isnull().sum() > 0:
    merged_data['Weekend_Label'] = merged_data['Weekend_Label'].fillna("Unknown")

merged_data['Holiday_Label'] = merged_data['IsHoliday'].map({1: "Holiday", 0: "notHoliday"})
#print(merged_data['isWeekend'] == 1)
# 检查是否有空值，并修复（如必要）
if merged_data['Holiday_Label'].isnull().sum() > 0:
    merged_data['Holiday_Label'] = merged_data['Holiday_Label'].fillna("Unknown")

# 绘制平日和周末酒店价格的趋势
plt.figure(figsize=(12, 6))
sns.lineplot(
    data=merged_data,
    x="Date",
    y="Hotel_Prices",
    hue="Weekend_Label",  # 按 'Weekend_Label' 分组绘制
    errorbar=None
)

# 图表美化
plt.title("Hotel Prices Trend: Weekday vs Weekend", fontsize=16)
plt.xlabel("Date", fontsize=12)
plt.ylabel("Hotel Prices (Normalized)", fontsize=12)
plt.legend(title="Day Type", fontsize=10)
plt.xticks(rotation=45)
plt.tight_layout()

# 显示图表
plt.show()

# 按假日和工作日绘制酒店价格变化的箱线图
plt.figure(figsize=(10, 6))
sns.boxplot(x="Weekend_Label", y="Hotel_Prices", data=merged_data, order=["Weekday", "Weekend"])

# 图表美化
plt.title("Hotel Prices: Weekdays vs Weekends", fontsize=16)
plt.xlabel("Day Type", fontsize=12)
plt.ylabel("Hotel Prices (Normalized)", fontsize=12)

# 显示图表
plt.show()
# 按假日和工作日绘制酒店价格变化的箱线图
plt.figure(figsize=(10, 6))
sns.boxplot(x="Holiday_Label", y="Hotel_Prices", data=merged_data, order=["Holiday", "notHoliday"])

# 图表美化
#print((merged_data['IsHoliday'] == 1).head(20))
plt.title("Hotel Prices: Holiday vs notHoliday", fontsize=16)
plt.xlabel("Day Type", fontsize=12)
plt.ylabel("Hotel Prices (Normalized)", fontsize=12)

# 显示图表
plt.show()

# 9. 顯示合併後的數據
print("\n合併後的數據:")
print(merged_data.head())
